# Test Suite
---

# Setup

## Prepare Functionality

In [ ]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from db import connection, engine
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc, f1_score
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import helpers_new as fdn

In [ ]:
print('My pandas version is {}. Please use version 0.23.1'.format(pd.__version__))
print('My numpy version is {}. Please use version 1.13.1'.format(np.__version__))
# import sklearn
# print('The scikit-learn version is {}. Please use version 0.20.1'.format(sklearn.__version__))

In [ ]:
# Prepare Attributes
def cleanData(df, filters):
    if 'beschaffungsstelle_plz' in filters:
        df[['beschaffungsstelle_plz']] = df[['beschaffungsstelle_plz']].applymap(fdn.tonumeric)
    if 'gatt_wto' in filters:
        df[['gatt_wto']] = df[['gatt_wto']].applymap(fdn.unifyYesNo)
    if 'preis' in filters:
        df[['preis']] = df[['preis']].applymap(fdn.createPriceCategory)
    if 'anzahl_angebote' in filters:
        df[['anzahl_angebote']] = df[['anzahl_angebote']].applymap(fdn.tonumeric)
    if 'teilangebote' in filters:
        df[['teilangebote']] = df[['teilangebote']].applymap(fdn.unifyYesNo)
    if 'lose' in filters:
        df[['lose']] = df[['lose']].applymap(fdn.unifyYesNo)
    if 'varianten' in filters:
        df[['varianten']] = df[['varianten']].applymap(fdn.unifyYesNo)
    if 'auftragsart_art' in filters:
        vectorizer = CountVectorizer(binary=True)
        X = vectorizer.fit_transform(df['auftrags_art'].values)
        text_columns = vectorizer.get_feature_names()
        title_df = pd.DataFrame(X.todense(), columns=text_columns)
        df = pd.concat([df, title_df], axis=1)
        df = df.drop('projekt_titel', axis=1)
    return df

In [ ]:
def prepareForRun(df_pos, df_neg_all, filterAttributes):
    # What attributes the model will be trained by
    filters = ['Y', 'meldungsnummer', 'ausschreibung_cpv'] + filterAttributes
    df_ready_all = []
    for df_neg in df_neg_all:
        # Merge positive and negative df into one, only use selected attributes
        df_tmp = df_pos.append(df_neg, ignore_index=True)[filters].copy()
        # Clean the data of all selected attributes
        df_tmp = cleanData(df_tmp, filterAttributes)
        df_ready_all.append(df_tmp)
    return df_ready_all

In [ ]:
for idx, df in enumerate(dataFrame): # enum to get index
        run = shuffle(df) 
        unique_mn = run.meldungsnummer.unique()
        xUniqueTest, xUniqueTrain = train_test_split(unique_mn, test_size=test_size)
        xAndYTest = run[run['meldungsnummer'].isin(xUniqueTest)].copy()
        xAndYTrain = run[run['meldungsnummer'].isin(xUniqueTrain)].copy()
        xtest = xAndYTest.iloc[:, 1:]
        ytest = xAndYTest.iloc[:, 0]
        xtrain = xAndYTrain.iloc[:, 1:]
        ytrain = xAndYTrain.iloc[:, 0]
        print('Unique_mn {}'.format(len(unique_mn)))
        print('xUniqueTest {}'.format(len(xUniqueTest)))
        print('xUniqueTrain {}'.format(len(xUniqueTrain)))
        print('ytest {}'.format(len(ytest)))
        print(xtest)
        print('xtrain {}'.format(len(xtrain)))
        print('xAndYTest {}'.format(len(xAndYTest)))
        print('------')

In [ ]:
run = shuffle(dataFrame[1]) # X times an indendical df. Each will be shuffeled
unique_mn = run.meldungsnummer.unique()
xUniqueTest, xUniqueTrain = train_test_split(unique_mn, test_size=test_size)
xAndYTest = run[run['meldungsnummer'].isin(xUniqueTest)].copy()
xAndYTrain = run[run['meldungsnummer'].isin(xUniqueTrain)].copy()
xtest = xAndYTest.iloc[:, :]
ytest = xAndYTest.iloc[:, 0]
xtrain = xAndYTrain.iloc[:, :]
ytrain = xAndYTrain.iloc[:, 0]
duplicates = 0

for x in xtest.values:
    for y in xtrain.values:
       # print('{} == {}'.format(x[1], y[1]))
        if x[1] == y[1]:
            duplicates += 1
print(duplicates)

In [ ]:
def runDecisionTree(dataFrame, trees, depth, test_size):
    xTests = [];
    yTests = [];
    for idx, df in enumerate(dataFrame): # enum to get index
        run = shuffle(df) # X times an indendical df. Each will be shuffeled
        
        unique_mn = run.meldungsnummer.unique()
    
        xUniqueTest, xUniqueTrain = train_test_split(unique_mn, test_size=test_size)
    
        xAndYTest = run[run['meldungsnummer'].isin(xUniqueTest)].copy()
        xAndYTrain = run[run['meldungsnummer'].isin(xUniqueTrain)].copy()
        
        xtest = xAndYTest.iloc[:, 2:]
        ytest = xAndYTest.iloc[:, 0]
        
        xtrain = xAndYTrain.iloc[:, 2:]
        ytrain = xAndYTrain.iloc[:, 0]
        # train the model on training sets
        #clf = tree.DecisionTreeClassifier()
        clf = RandomForestClassifier(n_estimators=trees, max_depth=depth, random_state=0)
        clf = clf.fit(xtrain, ytrain)
        print(clf.score(xtrain, ytrain))    # TODO: Explain
        # predict on the test sets
        prediction = clf.predict(xtest)
        # pandas.series to data frame
        df_ytest = ytest.to_frame()
        # add run number to df
        df_ytest['run'] = idx
        xtest['run'] = idx
        # add prediction to df
        df_ytest['prediction']= prediction
        # add result of run to df
        df_ytest['correct'] = df_ytest['prediction']==df_ytest['Y']
        # add run to run arrays
        xTests.append(xtest)
        yTests.append(df_ytest)
    return xTests, yTests

In [ ]:
def getAccuracies(dfys):  
    res = pd.DataFrame(columns=['accuracy', 'f1_score', 'fn rate'])
    for dfy in dfys:
        acc = round(accuracy_score(dfy.Y, dfy.prediction), 4)
        f1 = round(f1_score(dfy.Y, dfy.prediction), 4)
        cm = confusion_matrix(dfy.Y, dfy.prediction)
        fnr = round(cm[1][0] / (cm[1][1] + cm[1][0]), 4)
        res.loc[len(res)] = [ acc*100, f1*100, fnr*100 ] # add row to end of df, *100 for better % readability
    return res

In [ ]:
def getConfusionMatices(dfys):  
    res = pd.DataFrame(columns=['tn', 'tp', 'fp', 'fn'])
    for dfy in dfys:
        # ConfusionMatrix legende:
        # [tn, fp]
        # [fn, tp]
        cm = confusion_matrix(dfy.Y, dfy.prediction)
        res.loc[len(res)] = [ cm[0][0], cm[1][1], cm[0][1], cm[1][0] ]
    res.loc['sum'] = res.sum() # Summarize each column
    return res

## Choose Institution & Get Data
Only needs to be done once per bidder

In [ ]:
# Choose a bidder to train a model for (number of positive marked after the name)

#anbieter = 'Arnold AG' #1006
#anbieter = 'Alpiq AG' #827
#anbieter = 'Siemens AG' #641
#anbieter = 'Marti AG' #621
#anbieter = 'Swisscom' #602
#anbieter = 'Axpo AG' #577
#anbieter = 'Hewlett-Packard' #155
#anbieter = 'BG Ingénieurs Conseils' SA #151
#anbieter = 'Pricewaterhousecoopers' # 92
anbieter = 'Helbling Beratung + Bauplanung AG' #67
#anbieter = 'Ofrex SA' #40
#anbieter = 'PENTAG Informatik AG' #40
#anbieter = 'Wicki Forst AG' #30
#anbieter = 'T-Systems Schweiz' #30
#anbieter = 'Bafilco AG' #20
#anbieter = '4Video-Production GmbH' #20
#anbieter = 'Widmer Ingenieure AG' #10
#anbieter = 'hmb partners AG' #10
#anbieter = 'Planmeca' #5
#anbieter = 'K & M Installationen AG' #5

In [ ]:
select_anbieter = (
    "anbieter.anbieter_id, "
    "anbieter.institution as anbieter_institution, "
    "cpv_dokument.cpv_nummer as anbieter_cpv, "
    "ausschreibung.meldungsnummer"
)
# anbieter_CPV are all the CPVs the Anbieter ever won a procurement for. So all the CPVs they are interested in. 
select_ausschreibung = (
    "anbieter.anbieter_id, "
    "auftraggeber.institution as beschaffungsstelle_institution, "
    "auftraggeber.beschaffungsstelle_plz, "
    "ausschreibung.gatt_wto, "
    "ausschreibung.sprache, "
    "ausschreibung.auftragsart_art, "
    "ausschreibung.lose, "
    "ausschreibung.teilangebote, "
    "ausschreibung.varianten, "
  #  "ausschreibung.titel, " TODO: Projektbeschrieb
    "ausschreibung.bietergemeinschaft, "
    "cpv_dokument.cpv_nummer as ausschreibung_cpv, "
    "ausschreibung.meldungsnummer as meldungsnummer2"
)

def getResponses(select_anbieter, select_ausschreibung, bidder, response):
    resp = '=';
    if (not response):
        resp = '!='
    query = """SELECT * FROM (SELECT {} from ((((((beruecksichtigteanbieter_zuschlag
            INNER JOIN zuschlag ON zuschlag.meldungsnummer = beruecksichtigteanbieter_zuschlag.meldungsnummer)
            INNER JOIN anbieter ON beruecksichtigteanbieter_zuschlag.anbieter_id = anbieter.anbieter_id)
            INNER JOIN projekt ON zuschlag.projekt_id = projekt.projekt_id)
            INNER JOIN auftraggeber ON projekt.auftraggeber_id = auftraggeber.auftraggeber_id)
            INNER JOIN ausschreibung ON projekt.projekt_id = ausschreibung.projekt_id)
            INNER JOIN cpv_dokument ON cpv_dokument.meldungsnummer = ausschreibung.meldungsnummer)
            WHERE anbieter.institution {} "{}" ) anbieter
        JOIN (SELECT {} from ((((((beruecksichtigteanbieter_zuschlag
            INNER JOIN zuschlag ON zuschlag.meldungsnummer = beruecksichtigteanbieter_zuschlag.meldungsnummer)
            INNER JOIN anbieter ON beruecksichtigteanbieter_zuschlag.anbieter_id = anbieter.anbieter_id)
            INNER JOIN projekt ON zuschlag.projekt_id = projekt.projekt_id)
            INNER JOIN auftraggeber ON projekt.auftraggeber_id = auftraggeber.auftraggeber_id)
            INNER JOIN ausschreibung ON projekt.projekt_id = ausschreibung.projekt_id)
            INNER JOIN cpv_dokument ON cpv_dokument.meldungsnummer = ausschreibung.meldungsnummer)
            WHERE anbieter.institution {} "{}"
            ) ausschreibung ON ausschreibung.meldungsnummer2 = anbieter.meldungsnummer;
    """.format(select_anbieter, resp, bidder, select_ausschreibung, resp, bidder)
    # pd.read_sql(query, connection);
    return query.replace('\n', '')


getResponses(select_anbieter, select_ausschreibung, anbieter, True)

In [ ]:
select_anbieter = (
    "anbieter.anbieter_id, "
    "anbieter.institution as anbieter_institution, "
    "cpv_dokument.cpv_nummer as anbieter_cpv, "
    "ausschreibung.meldungsnummer"
)
# anbieter_CPV are all the CPVs the Anbieter ever won a procurement for. So all the CPVs they are interested in. 
select_ausschreibung = (
    "anbieter.anbieter_id, "
    "auftraggeber.institution as beschaffungsstelle_institution, "
    "auftraggeber.beschaffungsstelle_plz, "
    "ausschreibung.gatt_wto, "
    "ausschreibung.sprache, "
    "ausschreibung.auftragsart_art, "
    "ausschreibung.lose, "
    "ausschreibung.teilangebote, "
    "ausschreibung.varianten, "
  #  "ausschreibung.titel, " TODO: Projektbeschrieb
    "ausschreibung.bietergemeinschaft, "
    "cpv_dokument.cpv_nummer as ausschreibung_cpv, "
    "ausschreibung.meldungsnummer as meldungsnummer2"
)
# Get all positive and negative responses
responses_positive, full_negative = fdn.createAnbieterDf(select_anbieter, select_ausschreibung, anbieter)
responses_positive.head(3)

# Tuning

***
***ToDo:***
* TEST WITH BIDDERS OF DIFFERENT SIZE!
* Add Auftrags_art as catagory!!
* Create function to read all which tenderings are FPs / FNs
* Create a n times n input for all attributes
* Test with Random Forest
* Enable better auto Evaluation Feedback. Maybe some Graphs or something
* Prepare (and test with) more Attributes
* Test Model only with Tenderings from the same ream / CPV category
* ("ctrl + F" all TODOs in this file)
* (Take a look at warning when tree is run)

***Fragen:***
* Müssen wir die Freihänder beachten?
* Spielen Attribute des Zuschlags überhaupt eine Rolle?
* Welche Attribute wollen wir noch anschauen / einbringen?
* Wenn wir nur den CPV verwenden, ist die Anwendung besser als ein normaler Filter?
* Könnte unser Algorithmus einen Bias haben, da wir mehrer CPV miteinander kombinieren, wenn wir die Tables laden?

***Notes Zwischenpräsentation***
* Design Science Vorgehen erweitern
* Wie kann man Vertrauen in einen Recommendation Algorithmus schaffen? --> Möglichst tansparent dem User sagen, was wird in Betracht gezogen
* Kann man die Wichtigkeit von einzelnen Attributen werten? / Gewichten
* Der Algorithmus muss "resetbar" sein
* Nick Lüthi: Transparenz und Einsicht in Algorithmus? Wie kommt er auf das Ergebnis? Der Markt muss genung Vertauen in den Vorgang haben.

***

In [ ]:
# ratio that the positive and negative responses have to each other
positive_to_negative_ratio = 2/3
# Percentage of training set that is used for testing (Recommendation of at least 25%)
test_size = 0.25

In [ ]:
# train n different models on n different (reproducable) sample sizes
runs = 20

In [ ]:
# Attributes ready for use: 'beschaffungsstelle_plz', 'gatt_wto', 'lose', 'teilangebote', 'varianten'

# Next focus: 'beschaffungsstelle_institution', 'titel', 'sprache', 'auftragsart_art' <-- AUFTRAGSART!

# ???: 'Preis', 'anzahl_angebote'

#attributes = [ 'gatt_wto', 'lose', 'teilangebote', 'varianten']
attributes = [ 'gatt_wto', 'lose', 'teilangebote', 'varianten', 'beschaffungsstelle_plz']
attributes = []

In [ ]:
# Tune Random Forest Parameter
trees = 100
depth = 14

# Model Creation

In [ ]:
# create the chosen amount reproducable samples for negative DataFrames with the ratio definded above
responses_negative_all = fdn.createNegativeResponses(
    full_negative,
    len(responses_positive),
    runs,
    positive_to_negative_ratio)

# Assign positive and negative lables to both DFs
responses_positive['Y'] = 1
for df in responses_negative_all:
    df['Y'] = 0

In [ ]:
dataFrame = prepareForRun(responses_positive, responses_negative_all, attributes)
responses_positive.head()
print(attributes)
dataFrame

In [ ]:
def unique_test_train_split(dataFrame, test_size):

    for idx, df in enumerate(dataFrame): # enum to get index
        df
        run = shuffle(df)
        # Put responses in one arry and all desired properties in another
        #y = run.iloc[:, 0]
        #x = run.iloc[:, 1:] # Every column but the first
    
        unique_mn = run.meldungsnummer.unique()
    
        xUniqueTest, xUniqueTrain = train_test_split(unique_mn, test_size=test_size)
    
        xAndYTest = run[run['meldungsnummer'].isin(xUniqueTest)].copy()
        xAndYTrain = run[run['meldungsnummer'].isin(xUniqueTrain)].copy()
        
        xTest = xAndYTest.iloc[:, 1:]
        yTest = xAndYTest.iloc[:, 0]
        
        xTrain = xAndYTrain.iloc[:, 1:]
        yTrain = xAndYTrain.iloc[:, 0]
        
        return yTest

    #total_number = len(unique_mn)
    
    #number_of_test_items = int(total_number * test_size)
    
    #test_items = []
    
    #for i in range(number_of_test_items):
    #    random.shuffle(unique_mn)
    
# unique_test_train_split(dataFrame, test_size)

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide some "slice of copy" warnings
xTests, yTests = runDecisionTree(prepareForRun(responses_positive, responses_negative_all, attributes), trees, depth, test_size)

# Evaluation

In [ ]:
print(attributes)
pd.concat([getConfusionMatices(yTests), getAccuracies(yTests)], axis=1, sort=False)

In [ ]:
# get(tp/tn, run) --> no run: get all runs
# get all IDs of FPs and FNs in list
# get corresponding attributes from outer list: CPV, CPV-description, title of tendering
# for each, add to DF
xTests[0].loc[1066,:]